In [92]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
city_popu=pd.read_hdf('./Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])


import os
dfo=pd.DataFrame()

for f in os.listdir('./DataSource/Haidilao/')[-6:]:
    # if any(i in f for i in ['2021-12']):
    print(f)
    date=f.split('_')[-1].split('.')[0]
    d=pd.read_csv('./DataSource/Haidilao\%s' % f)
    dfo=pd.concat([dfo,d],axis=0)

old=pd.read_hdf(r'Processed_Data/haidilao.h5',key='1229')
new=pd.concat([old,dfo])
new=new.drop_duplicates()
new.to_hdf(r'Processed_Data/haidilao.h5',key='1229')

hdl.queue.2023-01-11.csv
hdl.queue.2023-01-12.csv
hdl.queue.2023-01-13.csv
hdl.queue.2023-01-14.csv
hdl.queue.2023-01-15.csv
hdl.queue.2023-01-16.csv


In [93]:
#clean data
df=new.copy()
df['WeekDay']=pd.to_datetime(df['排队日期']).dt.strftime('%A')
df['城市']=df['城市'].str.replace('市',"")
df=df.rename(columns={'排队日期':'dt'})

def calculate_paidui(x):
    small=0
    mid=0
    big=0

    if x['小桌排号']==0 or x['小桌叫号']==0:
        small=0
    else:
        small=(x['小桌排号']-x['小桌叫号'])*2
        if small<0:
            small=0

    if x['中桌排号']==0 or x['中桌叫号']==0:
        mid=0
    else:
        mid=(x['中桌排号']-x['中桌叫号'])*4
        if mid<0:
            mid=0
    
    if x['大桌排号']==0 or x['大桌叫号']==0:
        big=0
    else:
        big=(x['大桌排号']-x['大桌叫号'])*4
        if big<0:
            big=0
    
    return small+mid+big


df['排队人数']=df.apply(lambda x:calculate_paidui(x),axis=1)

c1=df['排队人数']<0
df.loc[c1,'排队人数']=0


c2=df['国家']=='CN'
df=df[c2]

c3=df['dt'].isin(['2021-08-13','2022-12-27'])
df=df[~c3]

c4=df['排队时间'].isin('11:30,19:30,23:30'.split(',')) #11:30,12:30,13:30,17:30,18:30,19:30,20:30,21:30,22:30,23:30
df=df[c4]

c3=df['dt'].str.contains('2022')
store_2022=df[c3][['门店Id','城市','门店名称']].drop_duplicates(subset='门店Id')

c4=df['dt'].str.contains('2021')
store_2021=df[c4][['门店Id','城市','门店名称']].drop_duplicates(subset='门店Id')

same_store=store_2021.merge(store_2022,on=['门店Id'],how='inner')
df=df[df['门店Id'].isin(same_store['门店Id'])]
Ncount=df.groupby(by=['门店Id'])['排队人数'].count().reset_index()

def workday(s):
    if s in ['Saturday','Sunday']:
        return 'Weekend'
    else:
        return 'Workday'
df['workday']=df['WeekDay'].apply(workday)


print(Ncount['排队人数'].max(),Ncount['排队人数'].min())

70 60


In [94]:
storelist=df[['城市','门店Id','门店名称']].drop_duplicates(subset='门店Id')
storelist

,城市,门店Id,门店名称
0,通化,500105,通化中东新天地店
1,石家庄,20401,勒泰店
2,石家庄,20403,东胜广场店
3,石家庄,20402,海悦天地店
4,石家庄,20404,万达广场店
...,...,...,...
8464,东莞,130415,卓为城店
8465,东莞,130416,星河城店
8548,西宁,133601,美伦城市广场店
8549,西宁,133602,国芳百货店


In [95]:
d1=pd.pivot_table(df,index=['城市','门店Id'],columns=['dt'],values='排队人数',aggfunc='sum')
d1.merge(storelist,on=['城市','门店Id'],how='left').to_clipboard(index=False)

In [96]:
#set 2021 
#用weekday hour 平均，加上周末hourly 平均

c5=df['dt'].str.contains('2021')
df_21=df[c5]
df_21=df_21.groupby(by=['workday','排队时间','城市','门店Id'])['排队人数'].mean().reset_index()
df_21=df_21.groupby(by=['城市','workday'])['排队人数'].sum().reset_index()

d_yichang=pd.pivot_table(df_21,index='城市',columns='workday',values='排队人数',aggfunc='sum').reset_index() #异常数据
d_yichang['weekendratio']=d_yichang['Weekend']/d_yichang['Workday']
city_delete=d_yichang[d_yichang['weekendratio']<3]['城市']

c6=df_21['城市'].isin(city_delete)
df_21=df_21[~c6]
df_21=df_21.rename(columns={'排队人数':'排队人数21'})
df_21=pd.pivot(df_21,index='城市',columns='workday',values='排队人数21').reset_index()
df_21['排队人数21']=df_21['Workday']*5+df_21['Weekend']*2
# df_21=df_21[['城市','排队人数21']]
df_21.columns=['城市', 'Weekend_21', 'Workday_21', '排队人数21']
city_delete

18     信阳
34     台北
38     吉林
91     扬州
97     新乡
116    榆林
131    洛阳
141    清远
148    滨州
177    荆门
186    西宁
188    许昌
189    贵港
203    郑州
225    香港
231    黄冈
Name: 城市, dtype: object

In [97]:
#set 2021 baselineNO COVID baseline
#用weekday hour 平均，加上周末hourly 平均

c5=df['dt'].str.contains('2021')
df_nocovid=df[c5]

citydate=pd.read_hdf('Processed_Data/amap.h5',key='noCOVIDdf')

df_nocovid['城市date']=df_nocovid['城市']+df_nocovid['dt']

df_nocovid=df_nocovid[df_nocovid['城市date'].isin(citydate['城市date'])]

df_nocovid=df_nocovid.groupby(by=['workday','排队时间','城市','门店Id'])['排队人数'].mean().reset_index()
df_nocovid=df_nocovid.groupby(by=['城市','workday'])['排队人数'].sum().reset_index()

d_yichang=pd.pivot_table(df_nocovid,index='城市',columns='workday',values='排队人数',aggfunc='sum').reset_index() #异常数据
d_yichang['weekendratio']=d_yichang['Weekend']/d_yichang['Workday']

c1=d_yichang['weekendratio']<3
c2=~(d_yichang['Weekend']>0)
c3=~(d_yichang['Workday']>0)


city_delete2=d_yichang[c1|c2|c3]['城市']

c6=df_nocovid['城市'].isin(city_delete2)
df_nocovid=df_nocovid[~c6]
df_nocovid=df_nocovid.rename(columns={'排队人数':'排队人数21'})
df_nocovid=pd.pivot(df_nocovid,index='城市',columns='workday',values='排队人数21').reset_index()
df_nocovid['排队人数21nocovid']=df_nocovid['Workday']*5+df_nocovid['Weekend']*2
df_nocovid=df_nocovid[['城市','排队人数21nocovid']]

city_delete2


0      三亚
10     咸阳
14     天津
24     新乡
26     昆明
32     济南
36     清远
44    石家庄
54     西宁
Name: 城市, dtype: object

In [117]:
from datetime import datetime,timedelta

td=datetime.strptime('2023-01-17','%Y-%m-%d') #tod


dtlist=[datetime.strftime(td-timedelta(days=i),"%Y-%m-%d") for i in range(1,8)]

# print(','.join(dtlist))

from datetime import datetime,timedelta
dtlist_lw=[datetime.strftime(td-timedelta(days=i+7),"%Y-%m-%d") for i in range(1,8)]

print(','.join(dtlist))

2023-01-16,2023-01-15,2023-01-14,2023-01-13,2023-01-12,2023-01-11,2023-01-10


In [118]:
c7=df['dt'].isin(dtlist_lw)
c8=~df['城市'].isin(city_delete)
df_22_lw=df[c7&c8]
df_22_lw=df_22_lw.groupby(by=['workday','排队时间','城市','门店Id'])['排队人数'].mean().reset_index()
df_22_lw=df_22_lw.groupby(by=['城市','workday'])['排队人数'].sum().reset_index()
df_22_lw=pd.pivot(df_22_lw,index='城市',columns='workday',values='排队人数').reset_index()
df_22_lw['排队人数']=df_22_lw['Workday']*5+df_22_lw['Weekend']*2
df_22_lw=df_22_lw[['城市','排队人数']].rename(columns={'排队人数':'排队人数lw'})
df_22_lw

workday,城市,排队人数lw
0,三亚,7750.0
1,上海,25048.0
2,上饶,240.0
3,东莞,10128.0
4,东营,288.0
...,...,...
216,黄石,218.0
217,黔东南苗族侗族自治州,18.0
218,黔西南布依族苗族自治州,204.0
219,齐齐哈尔,62.0


In [119]:
#make comparable data

c7=df['dt'].isin(dtlist)
c8=~df['城市'].isin(city_delete)
df_22=df[c7&c8]
df_22.loc[df_22['dt']=='2023-01-02','workday']='Weekend'
df_22=df_22.groupby(by=['workday','排队时间','城市','门店Id'])['排队人数'].mean().reset_index()
df_22=df_22.groupby(by=['城市','workday'])['排队人数'].sum().reset_index()

df_22=pd.pivot(df_22,index='城市',columns='workday',values='排队人数').reset_index()

df_22['排队人数']=df_22['Workday']*5+df_22['Weekend']*2
df_22=df_22[['城市','排队人数','Weekend', 'Workday']]

hdl=df_22.merge(df_21,on='城市',how='left').merge(df_22_lw,on='城市',how='left')

hdl['排队_asof21']=hdl['排队人数']/hdl['排队人数21']

hdl=hdl.sort_values(by='排队人数21',ascending=False)

c1=hdl['排队_asof21']<10
c2=hdl['排队_asof21']>0.1
hdl=hdl[c1&c2]

d=hdl.copy()
d=d[['城市','排队_asof21']].reset_index(drop=True).rename(columns={'排队_asof21':'hdl'})
d['date']='2023-01-09'
d.to_hdf('Processed_Data/city_compare.h5',key='hdl')


hdl['排队_wow']=hdl['排队人数']/hdl['排队人数lw']-1

hdl['Weekend_asof21']=hdl['Weekend']/hdl['Weekend_21']
hdl['Workday_asof21']=hdl['Workday']/hdl['Workday_21']

c1=hdl['Weekend_asof21']<10
c2=hdl['Weekend_asof21']>0.1
hdl=hdl[c1&c2]

c1=hdl['Workday_asof21']<10
c2=hdl['Workday_asof21']>0.1
hdl=hdl[c1&c2]


In [120]:
# # covid rank vs. HDL vs.2021

# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

# df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')

# o=hdl.copy()

# hdl=hdl.sort_values(by='排队人数21',ascending=False).head(70).reset_index()
# o=o.merge(df_covid,on='城市',how='inner')

# o=o[['城市', 'progress_2','排队_asof21']]

# o['size']=np.nan
# o['group']='其他'

# c1=o['城市'].isin(dafeng_city)
# o.loc[c1,'group']='本波疫情基本结束'

# o=o.sort_values(by='group',ascending=False)
# o.to_clipboard(index=False,header=None)
# print('vs. 2021')


In [131]:
# 收入 vs. HDL vs.2021

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')

o=hdl.merge(city_popu,on='城市',how='inner')

o=o.sort_values(by='常住人口',ascending=False).head(70)


o=o[o['排队_asof21']<2][['城市', '人均可支配收入','排队_asof21']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by=['group','人均可支配收入'],ascending=False)
o.to_clipboard(index=False,header=None)
print('vs. 2021')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
vs. 2021


In [132]:
#comparison Data
c8=~df['城市'].isin(city_delete)
c9=df['dt'].astype('str')>='2022-12-28'
df_con=df[c8&c9]

df_con=df_con.groupby(by=['dt','城市'])['排队人数'].sum().reset_index()
df_con['排队人数_last7d_mean']=df_con.groupby('城市')['排队人数'].transform(lambda x: x.rolling(7, 7).mean())
df_con=df_con.merge(df_nocovid,on='城市',how='left')


df_con['hdl']=df_con['排队人数_last7d_mean']/(df_con['排队人数21nocovid']/7)
c1=df_con['hdl']>0
df_con=df_con[c1]

df_con=df_con.groupby('dt').sum()
df_con['hdl']=df_con['排队人数_last7d_mean']/(df_con['排队人数21nocovid']/7)
df_con.reset_index()[['dt','hdl']].to_clipboard()
df_con.reset_index()[['dt','hdl']]

,dt,hdl
0,2023-01-03,0.937803
1,2023-01-04,0.956454
2,2023-01-05,0.968747
3,2023-01-06,0.997097
4,2023-01-07,0.585518
5,2023-01-08,0.460136
6,2023-01-09,0.443271
7,2023-01-10,0.467850
8,2023-01-11,0.505075
9,2023-01-12,0.546345


In [130]:
# 收入 vs. HDL wow

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')

o=hdl.merge(city_popu,on='城市',how='inner')

o=o.sort_values(by='常住人口',ascending=False).head(70)


o=o[o['排队_wow']<2][['城市', '人均可支配收入','排队_wow']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by=['group','人均可支配收入'],ascending=False)
o.to_clipboard(index=False,header=None)
print('vs. lw')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
vs. lw


In [125]:
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','hdl')


d1=pd.read_hdf('Processed_Data/tablehead.h5','hdl')

d2=pd.pivot_table(df[df['dt'].astype('str')>'2023-01-09'],index=['城市'],columns=['dt','排队时间'],values='排队人数',aggfunc='sum').reset_index()
d2=d2.set_index('城市')
d2.reindex(d1['城市']).to_clipboard()


# # d1=pd.read_clipboard()
# # d1.to_hdf('Processed_Data/tablehead.h5','heytea')

# d1=pd.read_hdf('Processed_Data/tablehead.h5','heytea')
# d2=pd.pivot_table(ht[ht['date'].astype('str')>'2023-01-02'],index='城市',columns='date',values='等待杯量',aggfunc='mean').reset_index()
# d1.merge(d2,left_on='城市',right_on='城市',how='left').to_clipboard(index=False)